In [1]:
import pandas as pd
import os
import numpy as np
from bokeh.layouts import gridplot

url_2 = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
regional_file_name = "regions.csv"

file_name = 'WFPVAM_FoodPrices_version4_Retail.csv'

# Add region and datetime cols.
df = pd.read_csv(file_name)
df['date'] = pd.to_datetime(df.date, format='%Y-%m')

country = 'Uganda'

In [2]:
# only_bread.plot(x='date', y='Diff')

def datetime(x):
    return np.array(x, dtype=np.datetime64)


from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool


# COLORS
from bokeh.palettes import Dark2_5 as palette
import itertools


colors = itertools.cycle(palette)

# Question 1

Are there any food prices that are show negative/positive correlation, and is this correlation present throughout the years, or perhaps only in certain period? Can you perhaps detect possible ingredients of a certain other food product?

In [12]:
# Each product in a graph for each country.

df_country = df.loc[df.adm0_name == country]
plots = []
country_groups = df_country.groupby(['mkt_name'])

# allow for interactive hover tool
# Import the ColumnDataSource
from bokeh.models import ColumnDataSource



for country_group, country_row in country_groups:
    plot = figure(x_axis_type="datetime", title="Products in {}".format(country_group), tools='hover,pan,wheel_zoom,box_zoom,reset')
    product_groups = country_row.groupby(['cm_name'])
    for (group, row), color in zip(product_groups, colors):
        # create CDS to enable dynamic hovering
        row_cds = ColumnDataSource(row)
        # set source to CDS
        plot.line('date', 'Diff', color=color, legend=group, source=row_cds)
    plot.legend.click_policy = "hide"
    plot.legend.location = "top_right"
    hover = plot.select(dict(type=HoverTool))
    # set preffered info for hovertool to show
    hover.tooltips = [
        # need to fix daytime again now....
        ("date", "@date"),
        ("product", "@cm_name"),
        ("country", "@adm0_name")
        ]
    
    plots.append(plot)

show(gridplot([plots]))

# Question 2
Do countries in similar regions, also show similar price differences? And if differences occur, can you find a potential explanation?

In [4]:
# Each product in a graph for each country.


region_df = pd.read_csv(regional_file_name)
region_df.rename(columns={'name': 'adm0_name'}, inplace=True)
new_regions = region_df.loc[:, ['adm0_name', 'sub-region']]

df_regions = pd.merge(df, new_regions, on='adm0_name', how='left')
df = df_regions.copy()

In [5]:
y_axis = 'Diff'


plots = []
product_groups = df.groupby(['sub-region', 'cm_name', 'date'])[y_axis].mean().reset_index()
group = product_groups.groupby('sub-region')

for gr1, row1 in group:
    plot = figure(x_axis_type="datetime", title="Avg Products in {}".format(gr1), tools='hover,pan,wheel_zoom,box_zoom,reset')
    product_groups = row1.groupby(['cm_name'])
    for (gr2, row2), color in zip(product_groups, colors):
        plot.line(row2['date'], row2[y_axis], color=color, legend=gr2)
    plot.legend.click_policy = "hide"
    plot.legend.location = "top_right"
    hover = plot.select(dict(type=HoverTool))
    hover.tooltips = [
        ("Product", "@cm_name"),
        ("Value", "$y"),
        ]
    
    plots.append(plot)
show(gridplot([plots]))

# Question 3
Can we see any correlations between other types of data, like: currency fluctuations, weather patterns, and/or refugee movements. Can we perhaps visualize refugee movements from the food price data? Do certain weather conditions influence market prices?